# Descrição Estatística

**OBJETIVO**: O objetivo deste notebook é mostrar os príncipais métodos de manipulação de dados utilizando o pandas e numpy

---

## Cabeçalho

Seleciona as bibliotecas que devem ser carregadas no jupyter

In [ ]:
import os
import requests
import zipfile

import numpy as np
import pandas as pd

from io import BytesIO
from pathlib import Path
from pyarrow.lib import ArrowInvalid

Configurações do Jupyter que devem ser aplicadas

In [ ]:
%config Completer.use_jedi = False

# pode ser "completo" ou "amostra"
TIPO_DADOS = "completo"

# checa se o notebook está sendo executado no google colab
GOOGLE_COLAB = "google.colab" in str(get_ipython())

# monta a pasta com os conteúdos
if GOOGLE_COLAB:
    from google.colab import drive
    drive.mount("/content/drive")

Configura os caminhos para diferentes dados

In [ ]:
# informa o caminho para a pasta de ciência de dados a partir do drive
# você pode deixar isso como vazio se você não tiver adicionado a pasta
CAMINHO_DRIVE = "Ciência de Dados"

# links para os dados a serem baixados diretamente da Web
LINKS_DADOS = {
    "amostra": {
        "escola.parquet": "https://drive.google.com/uc?id=1i51S1GKVqc-_5KgWsfXDc3Fnkf1y700y&export=download",
        "ideb.parquet": "https://drive.google.com/uc?id=1NXCGhtluNbd2Vccyof3Y-eaw1vHkvIAL&export=download",
        "turma.parquet": "https://drive.google.com/uc?id=1ks0lsbU5GXI6sbZRG9f8zGdzZqDRK8SG&export=download",
        "dm_escola.parquet": "https://drive.google.com/uc?id=1Wpw3r7s2mimwwi5MH3hj94qYKE_Yvkp1&export=download"
    },
    "completo": {
        "escola.parquet": "https://drive.google.com/uc?id=1_at50Wh4JJz1jR-hhsqutVaUorN0M4IL&export=download",
        "ideb.parquet": "https://drive.google.com/uc?id=183GP-MVohBC84NvHMLYcBQ4YRRCPHxYi&export=download",
        "turma.parquet": "https://drive.google.com/uc?id=1OSrAB3oAAX-9NNWZ7dG0mcWFOIcjWUs8&export=download",
        "dm_escola.parquet": "https://drive.google.com/uc?id=1Y2RGm678gVyi3na7l53fA9kAFe1r38fo&export=download"
    }
}

# Caminhos locais para pastas de dados
vPATH_NOTEBOOK = Path(os.path.dirname(os.path.realpath("__file__")))
vPATH_DADOS = vPATH_NOTEBOOK.parent.parent / f"dados/{TIPO_DADOS}"

Define funções a serem usadas somente no Jupyter

In [ ]:
def carrega_parquet(nome: str, pasta: str, **kwargs) -> pd.DataFrame:
    global GOOGLE_COLAB, CAMINHO_DRIVE, vPATH_DADOS, TIPO_DADOS
    if GOOGLE_COLAB and CAMINHO_DRIVE != "":
        return pd.read_parquet(f"drive/MyDrive/{CAMINHO_DRIVE}/dados/{TIPO_DADOS}/{pasta}/{nome}.parquet", **kwargs)
    elif GOOGLE_COLAB or not os.path.exists(vPATH_DADOS / f"{pasta}/{nome}.parquet"):
        return pd.read_parquet(LINKS_DADOS[TIPO_DADOS][f"{nome}.parquet"])
    else:
        try:
            return pd.read_parquet(vPATH_DADOS / f"{pasta}/{nome}.parquet", **kwargs)
        except ArrowInvalid:
            return pd.read_parquet(LINKS_DADOS[TIPO_DADOS][f"{nome}.parquet"])

Carrega as bases a serem utilizadas

In [ ]:
df = carrega_parquet("dm_escola", "datamart", filters=[("ANO", "=", 2020)])

---

## Medidas de Tendencia Central

As medidas de têndencia central buscam descrever os dados com o objetivo de encontrar o comportamento mais comum do mesmo. Isto é, por exemplo, se nós selecionássemos uma escola aleatoriamente dos nossos dados quais seriam as características mais prováveis da mesma?

Uma maneira de visualizar os dados de uma entidade por uma variável é por meio de um histograma, que mostrar a quantidade de pontos que estão dentro de uma faixa de valores. No exemplo abaixo nós vemos a quantidade de escolas para 10 faixas distintas de IDEB anos iniciais.

### Média

$\overline{x} = \frac{1}{n} \sum_{i=1}^{n} x_{i}$

### Mediana

$
m\left(x\right)
=\begin{cases}
  x_\frac{n+1}{2}                                    & n\text{ odd}\\
  \frac {1}{2}\left(x_{\frac{n}{2}} + x_{\frac{n}{2} + 1}\right) & n \text{ even}
\end{cases}
$

![title](https://www.varsitytutors.com/assets/vt-hotmath-legacy/hotmath_help/topics/interquartile/interquartile.gif)

### Moda

---

## Medidas de Dispersão

As medidas de dispersão buscam mapear a variação de valores dentro de uma distribuição, isto é, dado qual é o valor esperado de um determinado ponto, quanto dos nossos dados caem dentro de determinadas faixas de valores que divergem da medida de tendência central

### Amplitude

$A = MAX(x_i \text{ for i in n}) - MIN(x_i \text{ for i in n})$

### Variância

$v = \frac{1}{N-1} \sum_{i=1}^N (x_i - \overline{x})^2$

### Desvio-Padrão

$s = \sqrt{\frac{1}{N-1} \sum_{i=1}^N (x_i - \overline{x})^2}$

### Percentil

$P_i = \frac{i}{100}(n + 1)^{th}$ term

$Q_1 = \frac{1}{4}(n + 1)^{th}$ term

$Q_2 = \frac{1}{2}(n + 1)^{th}$ term

$Q_3 = \frac{3}{4}(n + 1)^{th}$ term

![title](https://www.mathsisfun.com/data/images/quartiles-a.svg)

### Distância inter-quartil

![title](https://cdn1.byjus.com/wp-content/uploads/2021/03/interquartile-range.png)

---

## Descrevendo os Dados

Com as medidas de tendência central e dispersão nós podemos descrever as características de uma população tendo em vista as variáveis que as descrevem. Nós já vimos algumas maneiras de fazer isso.

A primeira é pelo describe, que nos permite tirar rapidamente métricas quanto a distribuição da variável

A segunda é a medida de histograma. Um histograma é uma espécie de gráfico de barras que demonstra uma distribuição de frequências. No histograma, a base de cada uma das barras representa uma classe e a altura representa a quantidade ou frequência absoluta com que o valor de cada classe ocorre. Ao mesmo tempo, ele pode ser utilizado como um indicador de dispersão de processos.

É importante nós percebemos que geralmente a distribuição dos dados são modeladas por meio de distribuições. Uma distribuição representa a probabilidade de um ponto ter um valor determinado. Por exemplo, se eu selecionar uma escola aleatória e verificar seu IDEB, qual é a probabilidade de o valor está entre 5.9 e 6.1? Com base nessa informação nós podemos desenhar o gráfico.

Em geral a distribuição mais comum que discutimos é a distribuição normal que tem características similares ao do IDEB, mas outras distribuições como binomial e exponencial (QT_DESKTOP_ALUNO) também são bastantes comuns. Nós entraremos em detalhes com relação a distribuições nas aulas de inferência.

![title](https://miro.medium.com/max/1200/1*IdGgdrY_n_9_YfkaCh-dag.png)

A terceira é por meio de um gráfico de caixa. Este gráfico é um método utilizado para a análise exploratória de variáveis quantitativas ou ordinais. Ele é uma ferramenta capaz de fornecer diversas medidas estatísticas como mínimo, primeiro quartil, segundo quartil (mediana), terceiro quartil, o máximo e os possíveis outliers (valores discrepantes).

![title](https://www.monolitonimbus.com.br/wp-content/uploads/2019/11/boxplot_horiz.png)

Com base nos gráficos acima, você deve imaginar que é razoavelmente simples de descrever quais são as faixas de valores esperadas para um determinada população + variável:

---

## Medidas de Similaridade

Tão importante quanto ser capaz de averiguar características de uma população ou variável é conseguir compara-la com outras populações ou variáveis, ou ser capaz de comparar individuos de uma população.

Para este fim as medidas de similaridade (ou dissimilaridade) foram criadas. Elas buscam tomar as caracterícas de um indíviduo e calcular algum tipo de métrica que permita coloca-lo numa escala de comparação com outro indíviduo.

### Matriz de Dados

A maneira mais comum de representar dados é através da matriz de dados. Nós já a vimos como arrays de numpy ou data frames de pandas, na qual cada linha representa uma entidade e cada coluna uma característica

d variáveis

n linhas de dados

$
\begin{bmatrix}
    x_{11} & x_{12} & x_{13} & \dots  & x_{1n} \\
    x_{21} & x_{22} & x_{23} & \dots  & x_{2n} \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    x_{d1} & x_{d2} & x_{d3} & \dots  & x_{dn}
\end{bmatrix}
$

Essa representação matricial dos dados pode ser enxergada como uma posição num espaço de coordenadas cartesianas. As coordenadas cartesianas são pares ordenados do tipo P (x,y) que são representados no plano cartesiano, sendo P o ponto, x é o número real que é abscissa de P e y o número real que é a ordenada de P

![title](https://upload.wikimedia.org/wikipedia/commons/thumb/0/0e/Cartesian-coordinate-system.svg/1200px-Cartesian-coordinate-system.svg.png)

Apesar de nós geralmente pensarmos em coordenadas cartesianas como bi-dimensionais, nós também podemos fornecer coordenadas para pontos que sejam tridimensionais.

![title](https://upload.wikimedia.org/wikipedia/commons/1/15/Cartesian_coordinates_3D.svg)

Entretando, diferente de seres humanos, a algebra linear não se limita a 2D ou 3D. Desta forma uma matriz de dados pode ser considerada como um conjunto de entidades distribuídos num espaço N-dimensional, na qual N é o número de variáveis que caracterizam esta entidade

![title](https://www.researchgate.net/profile/Mario-Ruiz-Estrada/publication/228300293/figure/fig3/AS:302050933002264@1449025962929/The-diamond-coordinate-space-with-infinite-axes_Q320.jpg)

### Distâncias

Quando nós estamos falando de um espaço 2D ou 3D nós sabemos que existe uma distância entre dois pontos. Isso é claro pra todos, por exemplo, quando queremos sair de casa pra ir até algum lugar nós sabemos qual é a distância entre esses lugares.

Mas e quando estamos falando de uma entidade? Podemos calcular a distância entre entidade? A resposta é sim! Como uma entidade pode ser representada num espaço N-dimensional então é possível calcular a distância entre dois pontos.

Desta forma a primeira medida de similaridade que nós podemos trabalhar é justamente a distância. Quanto mais próximas duas entidades no espaço mais similares elas são. Vamos agora ver diferentes métodos de cálculo de distância

#### Distância Numérica

**Distância Euclidiana**

A mais popular das distâncias, e é aquela que usamos no dia a dia, calcula a menor distância que pode ser percorrida entre dois pontos

$d\left( p,q\right)   = \sqrt {\sum _{i=1}^{n}  \left(q_{i}-p_{i}\right)^2}$

![title](https://upload.wikimedia.org/wikipedia/commons/thumb/5/55/Euclidean_distance_2d.svg/1200px-Euclidean_distance_2d.svg.png)

**Distância de Manhattan**

A distância de Manhattan assume que o deslocamento entre dois pontos deve ser feito em direções verticais e horizontais, de tal forma que para percorrer entre dois pontos em x,y nós temos primeiro que percorrer a distância entre os valores de x (ou y) e depois os valores de y (ou x)

$d\left( p,q\right)   = \sum _{i=1}^{n}  |q_{i}-p_{i}|$

![title](https://ars.els-cdn.com/content/image/3-s2.0-B9780128038185000093-f09-18-9780128038185.jpg)

**Distância de Minkowski**

A distância de Minkowski é a forma genérica dos principais cálculos de distância numérica, na qual nós adicionamos um fator q que corresponde ao expoente do calculo aplicado. É fácil ver que se q = 2 temos a distância euclidiana e se q = 1 temos a distância de Manhattan

$d\left(d_{1}, d_{2}\right) = \left(\sum _{i=1}^{n} |d_{1i}-d_{2i}|^q\right)^\frac{1}{q}$

![title](https://www.researchgate.net/profile/Tomasz-Wachowicz/publication/257567989/figure/fig1/AS:671537870929921@1537118513078/Single-criterion-rooted-Minkowski-distance-16-for-various-root-coefficients-q.png)

#### Distância entre Ordinais

Variáveis ordinais são categorias que tem algum tipo de hierarquia numérica entre elas, por exemplo pequeno/médio/grande. No caso, para calcular a distância entre entidades nós atribuimos um valor numérico a cada valor (geralmente um rank) e depois normalizamos este com a diferença entre os valor máximos e mínimos possíveis (para um rank, isto é a apenas o número de valores)

#### Distância entre Binários

Para calcular a distância entre dois pontos no espaço tendo em vista as variáveis binarias nós fazemos o "match" variável a variável e depois calculamos o % de matches obtidos

$q = \text{# binary variables where obj1 = obj2 = 1}$

$r = \text{# binary variables where obj1 = 1 and obj2 = 0}$

$s = \text{# binary variables where obj1 = 0 and obj2 = 1}$

$t = \text{# binary variables where obj1 = obj2 = 0}$

$d(o_{1}, o_{2}) = \frac{r + s}{q + r + s + t}$

#### Distância entre Nominais

Variáveis nominais são aquelas que correspondem a categorias de valores. Para calcular a distância entre dois pontos no espaço tendo em vista as variáveis nominais nós fazemos o "match" variável a variável e depois calculamos o % de matches obtidos

$d(p, q)   = \frac{\text{n} - \text{m}}{\text{n}}$

#### Distância entre Strings

A distância entre strings corresponde a uma evolução dos cálculos de distância entre variáveis categóricas, na qual nós teremos que verificar algumas características dos textos apresentados para entender como podemos dizer que eles são distintos

**Distância de Hamming**

A distância de Hamming entre duas strings de igual comprimento é o número de posições nas quais os símbolos correspondentes são diferentes.

**Distância de Levenshtein**

É o número mínimo de edições de um caractere (inserções, exclusões ou substituições) necessárias para mudar uma palavra para outra.

**Distância de Damerau–Levenshtein**

É o número mínimo de operações (que consiste em inserções, exclusões ou substituições de um único caractere, ou transposição de dois caracteres adjacentes) necessárias para mudar uma palavra para outra

**Maior sequência comum**

Você receberá duas strings/padrões/sequências de objetos. Entre essas duas sequências/strings, você precisa encontrar a maior subsequência de elementos na mesma ordem presentes em ambas as strings ou padrões.

**Distância de Jaro–Winkler**

O algoritmo Jaro é uma medida de caracteres em comum, sendo não mais que a metade do comprimento da string mais longa em distância, com consideração para transposições. Winkler modificou esse algoritmo para apoiar a ideia de que as diferenças próximas ao início da string são mais significativas do que as diferenças próximas ao final da string. Jaro e Jaro-Winkler são adequados para comparar strings menores, como palavras e nomes.

**Comparação das métricas**

- Hamming: Permite substituição
- Levenshtein: Permite deleção, inserção e substituição
- Damerau–Levenshtein: Permite deleção, inserção, substituição e transposição de adjacentes
- Maior sequência comum: Permite inserção e deleção
- Jaro–Winkler: Apenas transposição

#### Distância de Atributos Mistos

Na maioria dos problemas nós teremos uma combinação entre variáveis numéricas e categóricas. Desta forma, é interessantes sermos capazes de calcular o valor das distâncias normalizando para todo o tipo possível de variável, conforme a fórmula abaixo.

$d\left(i, j\right) = \frac{\sum_{f=1}^{p} \delta_{ij}^{(f)}d_{ij}^{(f)}}{\sum_{f=1}^{p}  \delta_{ij}^{(f)}}$

$\delta_{ij}^{(f)} = 0$ Se uma das variáveis $x_{if}$ ou $x_{jf}$ está sem dado

$\delta_{ij}^{(f)} = 0$ Se $x_{if} = x_{jf} = 0$ e é um dado binário assimétrico

$\delta_{ij}^{(f)} = 1$ Para todos os outros casos

- Se f é numérico: $d_{ij}^{(f)} = \frac{|x_{if} - x_{jf}|}{max_{h}x_{hf} - min_{h}x_{hf}}$ (h -> para todos os pontos)
- Se f é nominal ou binário: $d_{ij}^{(f)} = 0$ se $x_{if} = x_{jf}$, caso contrário $d_{ij}^{(f)} = 1$
- Se f é ordinal: Pegue os ranks $r_{if}$ e calcule o valor $z_{if} = \frac{r_{if} - 1}{M_{f} - 1}$ e trabalhe a variável como numérica

### Matriz de Dissimilaridade

$
\begin{bmatrix}
    0 & d(1, 2) & d(1, 3) & \dots  & d(1, n) \\
    d(2, 1) & 0 & d(2, 3) & \dots  & d(2, n) \\
    d(3, 1) & d(3, 2) & 0 & \dots  & d(3, n) \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    d(n, 1) & d(n, 2) & d(n, 3) & \dots  & 0
\end{bmatrix}
$

Essa estrutura armazena uma coleção de distâncias que estão disponíveis para todos os pares de n objetos. Muitas vezes é representado por um tabela n por n

### Similaridade de Cosseno

$\cos ({\bf t},{\bf e})= {{\bf t} \cdot {\bf e} \over \|{\bf t}\| \|{\bf e}\|} = \frac{ \sum_{i=1}^{n}{{\bf t}_i{\bf e}_i} }{ \sqrt{\sum_{i=1}^{n}{({\bf t}_i)^2}} \sqrt{\sum_{i=1}^{n}{({\bf e}_i)^2}} }$

---

## Medidas de Relacionamento

As medidas de relacionamento buscam inferir quanto uma variável muda em relação a outra, e são formas simples de medir se duas variáveis estão relacionadas de alguma forma

### Covariância

$cov_{x,y}=\frac{\sum_{i=1}^{N}(x_{i}-\bar{x})(y_{i}-\bar{y})}{N-1}$

A covariância corresponde a média das diferenças entre o valor da variável e o seu valor médio. Se o valor de 1 for maior que a média e o valor de 2 também for maior que a média, então o produto será positivo, caso contrário, será negativo. Dessa forma a soma desses produtos nos permite inferir como eles variam entre si.

### Correlação Numérica

$r =
  \frac{ \sum_{i=1}^{n}(x_i-\bar{x})(y_i-\bar{y}) }{%
        \sqrt{\sum_{i=1}^{n}(x_i-\bar{x})^2}\sqrt{\sum_{i=1}^{n}(y_i-\bar{y})^2}}$

A correlação, por outro lado, é como se fosse a "covariância normalizada", isto é, é um valor cujo divisor vai permitir que o resultado sempre esteja entre -1 e 1. Desta forma fica mais fácil entender se duas variáveis estão relacionadas.

### Correlação Outros Tipos

A correlação entre outros tipos de variáveis também pode ser feita. Entretanto, esse cálculo requer fazer um teste de hipótese contra um distribuição. Então, neste primeiro momento, nós pularemos esse cálculo e depois de falar sobre testes e distribuições mostraremos como gera-lo.

---

## Exemplos

### Agrupamentos

Na maioria dos casos nós vamos fazer avaliações descritivas dos dados extraíndo métricas estatísticas não apenas nos dados completos mais em sub-grupos dos dados. Isso nos ajuda a entender como diferentes segmentos da minha população se comportam dado uma variável de interesse

### Análise de Correlação

Uma das primeiras avaliações que nós podemos fazer é avaliar quais são as variáveis que mais influenciam a nota do IDEB. Uma maneira simples de fazer isso é pegar as variáveis de maior correlação

### Análise de degrau

A análise de correlação funciona bem para variáveis numéricas, entretanto para variáveis categóricas e binárias isso é um pouco diferente. No caso de variáveis binárias uma maneira eficiente de avaliar as mesmas correlações que o que foi mostrado acima é por meio de uma análise de degrau, na qual nós calculamos as diferenças da média/mediana da variável alvo dado o binário 0 para o 1

### Analise de Similares

Vamos supor que você trabalhe para o estado do Amazonas e você quer entender quais escolas não estão performando como deveriam.

A primeira pergunta que você pode fazer é: Como eu sei como uma escola deveria estar performando? Uma das maneiras de fazer isso é por meio de uma análise de similaridade.

Por exemplo, suponhamos que você crie um grupo que são as TOP 1% de escolas com maior nota de IDEB. Fazendo uma escolha de variáveis você pode calcular a distância entre as demais escolas e essas escolas e avaliar as escolas mais "próximas" para determinar que essas estão sub-aproveitando seu IDEB